In [ ]:
import numpy as np
import math
from skimage import io, measure
from itertools import combinations
from skimage.draw import line
from collections import defaultdict
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
CONTOUR_THRESHOLD = 1

In [ ]:
def calc_right_angle_score(a, b, c):
    return abs(90 - calc_angle(a, b, c))

In [ ]:
def calc_angle(a, b, c):
    ang = math.degrees(math.atan2(c[1]-b[1], c[0]-b[0]) - math.atan2(a[1]-b[1], a[0]-b[0]))
    return ang + 360 if ang < 0 else ang

In [ ]:
def score_vertices(contour):
    vertex_scores = defaultdict(int)
    deltas = [5, 10, 20, 30]
    for delta in deltas:
        for i, point in enumerate(contour):
            point_prev = contour[(i - delta) % len(contour)]
            point_next = contour[(i + delta) % len(contour)]
            vertex_scores[i] += calc_right_angle_score(point_prev, point, point_next)
    return vertex_scores

In [ ]:
#TODO: check if two best angles are too close to each other and merge them if they do

In [ ]:
def choose_best_vertices(vertices):
    return sorted(vertices, key=vertices.get)[:20]

In [ ]:
def plot_vertices_on_contour(contour, vertices):
    plt.gca().set_aspect('equal')
    plt.plot(contour[:, 1], contour[:, 0], linewidth=2)
    for vertex in vertices:
        plt.scatter(contour[vertex, 1], contour[vertex, 0], s=100)
    plt.show()

In [ ]:
def score_two_vertices(point_a, point_b):
    y_a, x_a = point_a[0], point_a[1]
    y_b, x_b = point_b[0], point_b[1]
    rr, cc = line(y_a, x_a, y_b, x_b)
    score = 0
    for point in zip(rr, cc):
        if point in contour_set:
            score += 1
    return score

In [ ]:
def find_best_base_pair(contour, best_vertices):
    best_score = 0
    for a, b in combinations(best_vertices, 2):
        score = score_two_vertices(contour[a], contour[b])
        if score > best_score:
            best_score = score
            base_a, base_b = a, b
    return base_a, base_b

In [ ]:
images = [io.imread("{}/{}.png".format("data/set8", i)) for i in range(20)]
for img in images:
    contours = measure.find_contours(img, CONTOUR_THRESHOLD)
    contour = max(contours, key=lambda arr: len(arr))
    contour = np.rint(contour).astype('int')
    contour_set = {(y, x) for y, x in contour}

    vertex_scores = score_vertices(contour)
    best_vertices = choose_best_vertices(vertex_scores)
    base_a, base_b = find_best_base_pair(contour, best_vertices)
    plot_vertices_on_contour(contour, [base_a, base_b])